In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193668 sha256=30abe869c1ff4e3645339297c05ab2e579a23fa99004a6129fd2a978f3246e26
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [2]:
from surprise import Dataset 
import pandas as pd
from google.colab import drive
from surprise import Reader
from surprise import SVD
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

drive.mount('/gdrive') 
data = pd.read_csv('/gdrive/My Drive/DecisionTree/Capstone/Clean_Client_Spend_Data_v2.csv')
df = data[['CLIENT_ID', 'SUPPLIER_NORMALIZED_NAME', 'IBM_COMMODITY']]

Mounted at /gdrive


<ipython-input-2-cba90938e58a>:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/gdrive/My Drive/DecisionTree/Capstone/Clean_Client_Spend_Data_v2.csv')


In [3]:
import os
from surprise import BaselineOnly
from surprise import accuracy 
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader

In [5]:
df

,CLIENT_ID,SUPPLIER_NORMALIZED_NAME,IBM_COMMODITY
0,CLIENT3,LOGEVO,Uncategorized
1,CLIENT3,CAPGEMINI,IT Maintenance Services
2,CLIENT2,MICROSOFT,Software Licenses
3,CLIENT3,MEDIACOM,Advertising & Creative Agency Fee
4,CLIENT2,CUTRALE AUBURNDALE STO,Other Financial Services
...,...,...,...
715892,CLIENT4,COGNIZANT,Software Maintenance
715893,CLIENT1,NEWEDGE,Other Financial Services
715894,CLIENT2,BAIN AND COMPANY,Management Consulting
715895,CLIENT2,TAMPA JUICE SERVICE,Food Services


In [4]:
Transaction_count = df.groupby(['CLIENT_ID', 'IBM_COMMODITY', 'SUPPLIER_NORMALIZED_NAME']).size().reset_index(name='rating')
Transaction_count

,CLIENT_ID,IBM_COMMODITY,SUPPLIER_NORMALIZED_NAME,rating
0,CLIENT1,Advertising & Creative Agency Fee,ABIGGERCIRCLE,3
1,CLIENT1,Advertising & Creative Agency Fee,ADVERTISING HOUSE,1
2,CLIENT1,Advertising & Creative Agency Fee,AFP ADVANCED FOOD PRODUCTS,1
3,CLIENT1,Advertising & Creative Agency Fee,AFRICA DDB BRASIL PUBLICIDADE,3
4,CLIENT1,Advertising & Creative Agency Fee,AMAZON,2
...,...,...,...,...
39228,CLIENT4,Uniforms,ARAMARK,166
39229,CLIENT4,Vehicle & Equipment Acquisition - OEM,QDCL,12
39230,CLIENT4,Warehousing,COORDINATED RESOURCES,1
39231,CLIENT4,Warehousing,CROWN WORLDWIDE MOVING AND STORAGE,2


In [5]:
# normalize the rating by dividing by the total number of transactions per client and commodity
Transaction_count['total_transactions'] = Transaction_count.groupby(['CLIENT_ID', 'SUPPLIER_NORMALIZED_NAME'])['rating'].transform('sum')
Transaction_count['rating'] = Transaction_count['rating'] / Transaction_count['total_transactions']
print(Transaction_count)

      CLIENT_ID                          IBM_COMMODITY  \
0       CLIENT1      Advertising & Creative Agency Fee   
1       CLIENT1      Advertising & Creative Agency Fee   
2       CLIENT1      Advertising & Creative Agency Fee   
3       CLIENT1      Advertising & Creative Agency Fee   
4       CLIENT1      Advertising & Creative Agency Fee   
...         ...                                    ...   
39228   CLIENT4                               Uniforms   
39229   CLIENT4  Vehicle & Equipment Acquisition - OEM   
39230   CLIENT4                            Warehousing   
39231   CLIENT4                            Warehousing   
39232   CLIENT4                            Warehousing   

                 SUPPLIER_NORMALIZED_NAME    rating  total_transactions  
0                           ABIGGERCIRCLE  1.000000                   3  
1                       ADVERTISING HOUSE  0.200000                   5  
2              AFP ADVANCED FOOD PRODUCTS  1.000000                   1  
3      

In [6]:
Transaction_count['new_rating'] = round(Transaction_count['rating']*5)
Transaction_count['new_rating']

0        5.0
1        1.0
2        5.0
3        2.0
4        5.0
        ... 
39228    4.0
39229    5.0
39230    2.0
39231    1.0
39232    5.0
Name: new_rating, Length: 39233, dtype: float64

In [7]:
Transaction_count.drop(['rating','total_transactions'], axis=1)

,CLIENT_ID,IBM_COMMODITY,SUPPLIER_NORMALIZED_NAME,new_rating
0,CLIENT1,Advertising & Creative Agency Fee,ABIGGERCIRCLE,5.0
1,CLIENT1,Advertising & Creative Agency Fee,ADVERTISING HOUSE,1.0
2,CLIENT1,Advertising & Creative Agency Fee,AFP ADVANCED FOOD PRODUCTS,5.0
3,CLIENT1,Advertising & Creative Agency Fee,AFRICA DDB BRASIL PUBLICIDADE,2.0
4,CLIENT1,Advertising & Creative Agency Fee,AMAZON,5.0
...,...,...,...,...
39228,CLIENT4,Uniforms,ARAMARK,4.0
39229,CLIENT4,Vehicle & Equipment Acquisition - OEM,QDCL,5.0
39230,CLIENT4,Warehousing,COORDINATED RESOURCES,2.0
39231,CLIENT4,Warehousing,CROWN WORLDWIDE MOVING AND STORAGE,1.0


In [8]:
df=Transaction_count

In [9]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
encoder_client = LabelEncoder()
encoder_supplier = LabelEncoder()
df['CLIENT_ID'] = encoder_client.fit_transform(df['CLIENT_ID'])
df['label_SUPPLIER_NORMALIZED_NAME'] = encoder_supplier.fit_transform(df['SUPPLIER_NORMALIZED_NAME'])
print(df.head())

   CLIENT_ID                      IBM_COMMODITY  \
0          0  Advertising & Creative Agency Fee   
1          0  Advertising & Creative Agency Fee   
2          0  Advertising & Creative Agency Fee   
3          0  Advertising & Creative Agency Fee   
4          0  Advertising & Creative Agency Fee   

        SUPPLIER_NORMALIZED_NAME  rating  total_transactions  new_rating  \
0                  ABIGGERCIRCLE   1.000                   3         5.0   
1              ADVERTISING HOUSE   0.200                   5         1.0   
2     AFP ADVANCED FOOD PRODUCTS   1.000                   1         5.0   
3  AFRICA DDB BRASIL PUBLICIDADE   0.375                   8         2.0   
4                         AMAZON   1.000                   2         5.0   

   label_SUPPLIER_NORMALIZED_NAME  
0                             217  
1                             521  
2                             562  
3                             566  
4                            1102  


In [27]:
df.head()

,CLIENT_ID,IBM_COMMODITY,SUPPLIER_NORMALIZED_NAME,rating,total_transactions,new_rating,label_SUPPLIER_NORMALIZED_NAME
0,0,Advertising & Creative Agency Fee,ABIGGERCIRCLE,1.000,3,5.0,217
1,0,Advertising & Creative Agency Fee,ADVERTISING HOUSE,0.200,5,1.0,521
2,0,Advertising & Creative Agency Fee,AFP ADVANCED FOOD PRODUCTS,1.000,1,5.0,562
3,0,Advertising & Creative Agency Fee,AFRICA DDB BRASIL PUBLICIDADE,0.375,8,2.0,566
4,0,Advertising & Creative Agency Fee,AMAZON,1.000,2,5.0,1102


In [12]:
df.to_csv('Processed_recom.csv', index=False)

In [10]:
#Transform the dataframe to Surprise environment
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5))
data = Dataset.load_from_df(df[["CLIENT_ID", "label_SUPPLIER_NORMALIZED_NAME", "new_rating"]], reader)

In [11]:
trainset, testset= train_test_split(data, test_size = 0.25, random_state = 0)

In [15]:
trainset.rating_scale

(1, 5)

In [16]:
#supplier namw
trainset.n_items

20330

In [17]:
#client
trainset.n_users

4

In [18]:
trainset.n_ratings

29424

In [19]:
trainset.global_mean

3.2557436106579662

In [12]:
from surprise import KNNBasic
sim_options = {"name": "MSD", "user_based": False, "shrinkage": 100}  # no shrinkage
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [13]:
predictions = algo.test(testset)
RMSE = accuracy.rmse(predictions)

RMSE: 1.7723


In [22]:
#from surprise import KNNBaseline
#sim_options = {"name": "pearson_baseline", "user_based": True, "shrinkage": 100}  # no shrinkage
#algo1 = KNNBaseline(sim_options=sim_options)
#algo1.fit(trainset)

In [23]:
#predictions = algo1.test(testset)
#RMSE = accuracy.rmse(predictions)

In [24]:
#Top 10 suppliers for client 4 for example
#CLIENT_ID = 3
#items = algo.get_neighbors(CLIENT_ID, k=10)
#print(items)

[16, 18, 54, 57, 62, 75, 96, 98, 100, 104]


In [33]:
CLIENT_ID = [0,1,2,3]
for i in CLIENT_ID:
  items = algo.get_neighbors(i, k=10)
  print("client", i, ":", items)

client 0 : [66, 101, 151, 164, 174, 222, 231, 235, 240, 245]
client 1 : [4, 9, 13, 17, 19, 20, 25, 28, 34, 37]
client 2 : [112, 146, 273, 286, 297, 306, 361, 425, 483, 490]
client 3 : [16, 18, 54, 57, 62, 75, 96, 98, 100, 104]


In [31]:
#for recom in items:
#supplier = df.loc[df['label_SUPPLIER_NORMALIZED_NAME'] == recom, 'SUPPLIER_NORMALIZED_NAME'].iloc[0]
#print(f"Supplier: {supplier}")

Supplier: 1RA
Supplier: 1ST CALL PLUMBING
Supplier: 3M
Supplier: 3PL
Supplier: 4 LEVEL LOGISTICS
Supplier: 4SIDE SOLUTIONS
Supplier: 84.51 LLC
Supplier: 99JOBS DESENVOLVIMENTO HUMANO
Supplier: A AHMEDALLY AND
Supplier: A AND B ENGINEERING SERVICES


In [21]:
CLIENT_ID = [0,1,2,3]
for i in CLIENT_ID:
  items = algo.get_neighbors(i, k=10)
  for item in items:
    supplier = df.loc[df['label_SUPPLIER_NORMALIZED_NAME'] == item, 'SUPPLIER_NORMALIZED_NAME'].iloc[0]
    print("Client", i, f"Supplier: {supplier}")
    

Client 0 Supplier: 42ND STREET FILMS
Client 0 Supplier: A AMERICAN CONTAINER
Client 0 Supplier: AA ELECTRIC
Client 0 Supplier: AARHUS TECH
Client 0 Supplier: AB MAURI FLEISCHMANNS
Client 0 Supplier: ABIONOVA HYGIENE SERVICE
Client 0 Supplier: ABMS
Client 0 Supplier: ABRASICORP EQUIPMENT
Client 0 Supplier: ABSA GROUP
Client 0 Supplier: ABSORB SOFTWARE
Client 1 Supplier: 1000THINGS
Client 1 Supplier: 120 80
Client 1 Supplier: 1ACT AMERICAN TELECOM
Client 1 Supplier: 1S RARUS ITS
Client 1 Supplier: 2 FOR IT
Client 1 Supplier: 2 SN MENN
Client 1 Supplier: 24 7 COMMUNICATION SP ZOO
Client 1 Supplier: 24X7 TRANSLINES
Client 1 Supplier: 2B SNC DI BIONDA MARIA LUISA EC
Client 1 Supplier: 2MPACT
Client 2 Supplier: A AND S ENGINEERING SOLUTIONS LT
Client 2 Supplier: A2A ENERGIA
Client 2 Supplier: ACCESSABLE
Client 2 Supplier: ACE CANCELAS SIGLO XXI
Client 2 Supplier: ACHIEVE LEARNING NZ
Client 2 Supplier: ACMS
Client 2 Supplier: ACTIVV
Client 2 Supplier: ADEPTUM
Client 2 Supplier: ADVANCE IMPEX
